In [1]:
import os
import cv2
import numpy as np

In [2]:
data_dir = "D:\Project\Facial-mood\Facial_mood_Detection\Dataset"
model_file = 'mood_model.yml'
label_file = 'emotion_labels.npy'

face_size = (100, 100)

In [7]:
def train_mood_recognizer():
    print("[INFO] Preparing data....")

    faces = []
    labels = []
    emotions = []

    if not os.path.exists(data_dir):
        print(f"Error: dataset folder '{data_dir}' not found.")
        return

    emotion_folders = os.listdir(data_dir)

    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    label_id = 0

    for emotion_name in emotion_folders:
        emotion_path = os.path.join(data_dir, emotion_name)

        if not os.path.isdir(emotion_path):
            continue

        emotions.append(emotion_name)
        print(f"   - Processing emotion: {emotion_name} (ID: {label_id})")

        for image_name in os.listdir(emotion_path):
            image_path = os.path.join(emotion_path, image_name)

            img = cv2.imread(image_path)
            if img is None: continue

            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

            detected_faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

            for (x, y, w, h) in detected_faces:
                face_roi = gray[y:y+h, x:x+w]

                face_roi = cv2.resize(face_roi, face_size)

                faces.append(face_roi)
                labels.append(label_id)
                break

        label_id += 1

    if len(faces) == 0:
        print("Error: No faces found. Check your dataset path and structure.")
        return

    print(f"\n [INFO] Training on {len(faces)} images...")

    recognizer = cv2.face.FisherFaceRecognizer_create()
    recognizer.train(faces, np.array(labels))

    recognizer.save(model_file)
    np.save(label_file, emotions)

    print(f" [Sucess] Model saved to '{model_file}'")
    print(f" [INFO] Emotion Mapping: {emotions}")

print("Training function complete.")

Training function complete.


In [8]:
# Run the training
train_mood_recognizer()

[INFO] Preparing data....
   - Processing emotion: anger (ID: 0)
   - Processing emotion: contempt (ID: 1)
   - Processing emotion: disgust (ID: 2)
   - Processing emotion: fear (ID: 3)
   - Processing emotion: happy (ID: 4)
   - Processing emotion: sadness (ID: 5)
   - Processing emotion: surprise (ID: 6)

 [INFO] Training on 754 images...
 [Sucess] Model saved to 'mood_model.yml'
 [INFO] Emotion Mapping: ['anger', 'contempt', 'disgust', 'fear', 'happy', 'sadness', 'surprise']
